## Sistema de recomendación

In [1]:
import pandas as pd
import numpy as np
import json
import gzip
import os
import re

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
juegos = pd.read_csv('/content/drive/MyDrive/games.csv')
juegos.release_date = pd.to_datetime(juegos.release_date)
juegos['año'] = juegos.release_date.dt.year
juegos["free"] = juegos.price.apply(lambda x: "free to play".find(x.lower().strip()) >= 0)
patron = re.compile('[+|-]?[0-9]*\.[0-9]+')
juegos["price"] = juegos.price.apply(lambda x: float(x) if patron.fullmatch(x) else 0)
juegos["price"] = juegos.price.astype(float, errors='ignore')
print(juegos.head())
print(juegos.shape)

items = pd.read_csv('/content/drive/MyDrive/items.csv')
print(items.head())
print(items.shape)

reviews = pd.read_csv('/content/drive/MyDrive/reviews.csv')
print(reviews.head())
print(reviews.shape)

generos = pd.read_csv('/content/drive/MyDrive/genres.csv')
generos

   item_id                 app_name  \
0   761140      Lost Summoner Kitty   
1   643980                Ironbound   
2   670290  Real Pool 3D - Poolians   
3   767400                  弹炸人2222   
4   773570            Log Challenge   

                                              genres  price release_date  \
0  ['Action', 'Casual', 'Indie', 'Simulation', 'S...   4.99   2018-01-04   
1       ['Free to Play', 'Indie', 'RPG', 'Strategy']   0.00   2018-01-04   
2  ['Casual', 'Free to Play', 'Indie', 'Simulatio...   0.00   2017-07-24   
3                  ['Action', 'Adventure', 'Casual']   0.99   2017-12-07   
4            ['Action', 'Indie', 'Casual', 'Sports']   2.99   2018-01-04   

          developer     año   free  
0         Kotoshiro  2018.0  False  
1  Secret Level SRL  2018.0   True  
2      Poolians.com  2017.0   True  
3              彼岸领域  2017.0  False  
4          CGPills   2018.0  False  
(32133, 8)
   item_id  playtime_forever            user_id
0       10                 

,genre
0,Massively Multiplayer
1,Adventure
2,Web Publishing
3,RPG
4,Education
5,Free to Play
6,Action
7,Audio Production
8,Utilities
9,Animation and Modeling


# Recomendación item-item

## Codificar los géneros de los juegos

In [ ]:
def genres_encoder(juegos):
  for gen in generos.genre:
    juegos[gen] = juegos.genres.apply(lambda x: 1 if gen in x else 0)
  juegos.drop(columns=['genres'], inplace=True)

genres_encoder(juegos)
juegos.head()

,item_id,app_name,price,release_date,developer,año,free,Massively Multiplayer,Adventure,Web Publishing,...,Software Training,Casual,Strategy,Racing,Sports,Video Production,Photo Editing,Indie,Accounting,Early Access
0,761140,Lost Summoner Kitty,4.99,2018-01-04,Kotoshiro,2018.0,False,0,0,0,...,0,1,1,0,0,0,0,1,0,0
1,643980,Ironbound,0.00,2018-01-04,Secret Level SRL,2018.0,True,0,0,0,...,0,0,1,0,0,0,0,1,0,0
2,670290,Real Pool 3D - Poolians,0.00,2017-07-24,Poolians.com,2017.0,True,0,0,0,...,0,1,0,0,1,0,0,1,0,0
3,767400,弹炸人2222,0.99,2017-12-07,彼岸领域,2017.0,False,0,1,0,...,0,1,0,0,0,0,0,0,0,0
4,773570,Log Challenge,2.99,2018-01-04,CGPills,2018.0,False,0,0,0,...,0,1,0,0,1,0,0,1,0,0


## Eliminar columnas innecesarias

In [ ]:
juegos.drop(columns=["release_date","developer"],inplace=True)
juegos.free = juegos.free.astype(int)
juegos.head()

,item_id,app_name,price,año,free,Massively Multiplayer,Adventure,Web Publishing,RPG,Education,...,Software Training,Casual,Strategy,Racing,Sports,Video Production,Photo Editing,Indie,Accounting,Early Access
0,761140,Lost Summoner Kitty,4.99,2018.0,0,0,0,0,0,0,...,0,1,1,0,0,0,0,1,0,0
1,643980,Ironbound,0.00,2018.0,1,0,0,0,1,0,...,0,0,1,0,0,0,0,1,0,0
2,670290,Real Pool 3D - Poolians,0.00,2017.0,1,0,0,0,0,0,...,0,1,0,0,1,0,0,1,0,0
3,767400,弹炸人2222,0.99,2017.0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,773570,Log Challenge,2.99,2018.0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,1,0,0


In [ ]:
juegos.set_index("item_id",inplace=True)
juegos.head()

,app_name,price,año,free,Massively Multiplayer,Adventure,Web Publishing,RPG,Education,Free to Play,...,Software Training,Casual,Strategy,Racing,Sports,Video Production,Photo Editing,Indie,Accounting,Early Access
item_id,,,,,,,,,,,,,,,,,,,,,
761140,Lost Summoner Kitty,4.99,2018.0,0,0,0,0,0,0,0,...,0,1,1,0,0,0,0,1,0,0
643980,Ironbound,0.00,2018.0,1,0,0,0,1,0,1,...,0,0,1,0,0,0,0,1,0,0
670290,Real Pool 3D - Poolians,0.00,2017.0,1,0,0,0,0,0,1,...,0,1,0,0,1,0,0,1,0,0
767400,弹炸人2222,0.99,2017.0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
773570,Log Challenge,2.99,2018.0,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,1,0,0


In [ ]:
juegos_names = juegos.app_name
juegos_names.head()

item_id
761140        Lost Summoner Kitty
643980                  Ironbound
670290    Real Pool 3D - Poolians
767400                    弹炸人2222
773570              Log Challenge
Name: app_name, dtype: object

In [ ]:
juegos_data = juegos.drop(columns=["app_name"])
juegos_data.head()

,price,año,free,Massively Multiplayer,Adventure,Web Publishing,RPG,Education,Free to Play,Action,...,Software Training,Casual,Strategy,Racing,Sports,Video Production,Photo Editing,Indie,Accounting,Early Access
item_id,,,,,,,,,,,,,,,,,,,,,
761140,4.99,2018.0,0,0,0,0,0,0,0,1,...,0,1,1,0,0,0,0,1,0,0
643980,0.00,2018.0,1,0,0,0,1,0,1,0,...,0,0,1,0,0,0,0,1,0,0
670290,0.00,2017.0,1,0,0,0,0,0,1,0,...,0,1,0,0,1,0,0,1,0,0
767400,0.99,2017.0,0,0,1,0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,0
773570,2.99,2018.0,0,0,0,0,0,0,0,1,...,0,1,0,0,1,0,0,1,0,0


###Sparsity

In [ ]:
float(len(juegos_data.to_numpy().nonzero()[0]))/(juegos_data.shape[0]*juegos_data.shape[1])

0.1757607444060623

##Imputar nulos y escalar la columna año

In [ ]:
juegos_data.año.fillna(juegos_data.año.median(),inplace=True)
juegos_data.año = juegos_data.año.astype(int)
juegos_data.head()

,price,año,free,Massively Multiplayer,Adventure,Web Publishing,RPG,Education,Free to Play,Action,...,Software Training,Casual,Strategy,Racing,Sports,Video Production,Photo Editing,Indie,Accounting,Early Access
item_id,,,,,,,,,,,,,,,,,,,,,
761140,4.99,2018,0,0,0,0,0,0,0,1,...,0,1,1,0,0,0,0,1,0,0
643980,0.00,2018,1,0,0,0,1,0,1,0,...,0,0,1,0,0,0,0,1,0,0
670290,0.00,2017,1,0,0,0,0,0,1,0,...,0,1,0,0,1,0,0,1,0,0
767400,0.99,2017,0,0,1,0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,0
773570,2.99,2018,0,0,0,0,0,0,0,1,...,0,1,0,0,1,0,0,1,0,0


In [ ]:
from sklearn.preprocessing import minmax_scale

juegos_data.año = minmax_scale(juegos_data.año)
juegos_data.head()

,price,año,free,Massively Multiplayer,Adventure,Web Publishing,RPG,Education,Free to Play,Action,...,Software Training,Casual,Strategy,Racing,Sports,Video Production,Photo Editing,Indie,Accounting,Early Access
item_id,,,,,,,,,,,,,,,,,,,,,
761140,4.99,0.888889,0,0,0,0,0,0,0,1,...,0,1,1,0,0,0,0,1,0,0
643980,0.00,0.888889,1,0,0,0,1,0,1,0,...,0,0,1,0,0,0,0,1,0,0
670290,0.00,0.870370,1,0,0,0,0,0,1,0,...,0,1,0,0,1,0,0,1,0,0
767400,0.99,0.870370,0,0,1,0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,0
773570,2.99,0.888889,0,0,0,0,0,0,0,1,...,0,1,0,0,1,0,0,1,0,0


##Matriz de similaridad

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

similarities = cosine_similarity(juegos_data)
print(similarities.shape)

(32133, 32133)


In [ ]:
similarities = pd.DataFrame(similarities, index=juegos_names.index, columns=juegos_names.index)
similarities.head()

item_id,761140,643980,670290,767400,773570,772540,774276,774277,774278,768800,...,771810,767590,747320,769330,745400,773640,733530,610660,658870,681550
item_id,,,,,,,,,,,,,,,,,,,,,
761140,1.000000,0.209305,0.262041,0.639713,0.911465,0.922963,0.929121,0.921166,0.915102,0.837243,...,0.932058,0.718635,0.916626,0.301517,0.837243,0.898266,0.966867,0.824779,0.949862,0.934793
643980,0.209305,1.000000,0.603288,0.147716,0.200771,0.073961,0.113379,0.060658,0.038561,0.251495,...,0.066655,0.383138,0.076877,0.445377,0.251495,0.391985,0.216477,0.267228,0.140159,0.133124
670290,0.262041,0.603288,1.000000,0.310621,0.391769,0.153685,0.179561,0.096065,0.061070,0.490749,...,0.061132,0.549510,0.045237,0.638774,0.360703,0.490749,0.199201,0.383268,0.201707,0.191584
767400,0.639713,0.147716,0.310621,1.000000,0.710918,0.695801,0.479053,0.470438,0.465449,0.581457,...,0.562036,0.651571,0.475603,0.487841,0.892084,0.581457,0.575861,0.452802,0.585104,0.638715
773570,0.911465,0.200771,0.391769,0.710918,1.000000,0.834020,0.861856,0.841008,0.829824,0.797396,...,0.862018,0.799213,0.833990,0.450789,0.888629,0.797396,0.892315,0.750339,0.907975,0.862406


##Recomendaciones

In [ ]:
juego = 670290
similarities[juego].sort_values(ascending=False).drop(juego).head(10)

item_id
729580    1.000000
522660    0.999977
314000    0.932672
723610    0.923048
652980    0.923048
704040    0.923048
662700    0.923048
590280    0.923048
704340    0.923048
628610    0.923048
Name: 670290, dtype: float64

#   Recomendación user-item

In [ ]:
reviews.head()

,item_id,recommend,review,user_id,sentiment_analysis
0,1250,True,Simple yet with great replayability. In my opi...,76561197970982479,2
1,22200,True,It's unique and worth a playthrough.,76561197970982479,1
2,43110,True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479,2
3,251610,True,I know what you think when you see this title ...,js41637,2
4,227300,True,For a simple (it's actually not all that simpl...,js41637,2


In [ ]:
items.head()

,item_id,playtime_forever,user_id
0,10,6,76561197970982479
1,20,0,76561197970982479
2,30,7,76561197970982479
3,40,0,76561197970982479
4,50,0,76561197970982479


In [ ]:
res = reviews.copy()
res.head()

,item_id,recommend,review,user_id,sentiment_analysis
0,1250,True,Simple yet with great replayability. In my opi...,76561197970982479,2
1,22200,True,It's unique and worth a playthrough.,76561197970982479,1
2,43110,True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479,2
3,251610,True,I know what you think when you see this title ...,js41637,2
4,227300,True,For a simple (it's actually not all that simpl...,js41637,2


##Crear columna puntaje

<p>Para el puntaje se utilizará el promedio entre el analisis de sentimiento y la recomendación.

In [ ]:
res["puntaje"] = (res.sentiment_analysis+res.recommend)/2
res.drop(columns=["sentiment_analysis","recommend","review"],inplace=True)
res.head()

,item_id,user_id,puntaje
0,1250,76561197970982479,1.5
1,22200,76561197970982479,1.0
2,43110,76561197970982479,1.5
3,251610,js41637,1.5
4,227300,js41637,1.5


##Tabla pivote de usuarios y juegos

In [ ]:
res = res.pivot_table(index="user_id",columns="item_id",values="puntaje").fillna(0)
res.head()

item_id,10,20,30,40,50,60,70,80,130,220,...,512540,512630,514520,516040,520550,521340,521430,521570,521990,527340
user_id,,,,,,,,,,,,,,,,,,,,,
--000--,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
--ace--,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
--ionex--,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-2SV-vuLB-Kg,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-Azsael-,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
res.shape

(25458, 3682)

###Sparsity

In [ ]:
float(len(res.to_numpy().nonzero()[0]))/(res.shape[0]*res.shape[1])

0.0006027437209101664

##Enriquecimiento de la tabla pivote

<p>Para aumentar la información se utilizarán las horas de juego (playtime_forever). Se utilizará como límite superior 350 horas, ya que el 75% de los registros está en este rango. Las horas de juego se multiplicarán por el valor actual en la tabla pivote, de modo que aunque un juego se haya jugado por muchas horas, si tiene una mala reseña y no se recomienda, no se utilizaría en las sugerencias. El rango de este multiplicador es entre 0 y 10 (dividiendo las horas encontradas entre 35).

In [ ]:
cols = res.index
for item in res.columns:
  items_usr = items[items.item_id == item][["playtime_forever", "user_id"]].drop_duplicates(subset="user_id").set_index("user_id")["playtime_forever"]
  for user in [x for x in items_usr.index if x in cols]:
    hrs = items_usr[user]
    if hrs >= 350: res.loc[user,item] *= 10
    else: res.loc[user,item] *= hrs/35 if hrs is not None else 1
res.head()

item_id,10,20,30,40,50,60,70,80,130,220,...,512540,512630,514520,516040,520550,521340,521430,521570,521990,527340
user_id,,,,,,,,,,,,,,,,,,,,,
--000--,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
--ace--,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
--ionex--,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-2SV-vuLB-Kg,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-Azsael-,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,60.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Sparsity

In [ ]:
float(len(res.to_numpy().nonzero()[0]))/(res.shape[0]*res.shape[1])

0.016054229801721758

## Matriz de similaridad

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

users_similarities = cosine_similarity(res)
print(users_similarities.shape)

(25458, 25458)


In [ ]:
users_similarities = pd.DataFrame(users_similarities, index=res.index, columns=res.index)
users_similarities.head()

user_id,--000--,--ace--,--ionex--,-2SV-vuLB-Kg,-Azsael-,-Beave-,-GM-Dragon,-I_AM_EPIC-,-Kenny,-Mad-,...,zumpo,zunbae,zuzuga2003,zv_odd,zvanik,zwanzigdrei,zy0705,zynxgameth,zyr0n1c,zzoptimuszz
user_id,,,,,,,,,,,,,,,,,,,,,
--000--,1.000000,0.995462,0.995813,0.984868,0.000637,0.029654,0.006534,1.503772e-03,0.017799,0.088518,...,0.005672,0.964503,0.880678,0.099770,0.271284,0.675135,0.043446,0.003631,0.943295,0.009838
--ace--,0.995462,1.000000,0.997593,0.987116,0.000111,0.021570,0.004827,1.316109e-04,0.007113,0.089848,...,0.004524,0.964507,0.879442,0.081069,0.250269,0.674772,0.005194,0.002437,0.943946,0.007912
--ionex--,0.995813,0.997593,1.000000,0.988457,0.000265,0.024204,0.004376,1.897754e-03,0.003823,0.085719,...,0.004748,0.968978,0.880034,0.088341,0.256381,0.674023,0.008305,0.002845,0.945239,0.001766
-2SV-vuLB-Kg,0.984868,0.987116,0.988457,1.000000,0.000628,0.000000,0.003961,4.576169e-07,0.039297,0.078380,...,0.004472,0.955224,0.870309,0.080253,0.245554,0.665905,0.000004,0.000000,0.935353,0.000000
-Azsael-,0.000637,0.000111,0.000265,0.000628,1.000000,0.003738,0.063136,1.396339e-03,0.057538,0.138059,...,0.130291,0.001245,0.000003,0.070679,0.005579,0.004514,0.000000,0.003792,0.059964,0.000000


##Sugerencias

Para las sugerencias se utiliza el siguiente proceso:
- Se encuentran los usuarios más "similares".
- Para estos usuarios, se buscan sus juegos preferidos.
- Para estos juegos, se filtran los que no hayan sido jugados por el usuario a consultar.
- Luego, se ordenan y presentan.

In [ ]:
user = "--000--"
similares_user = users_similarities[user].sort_values(ascending=False).drop(user).head(5)
similares_user

user_id
76561198120465213    0.996762
76561198076299728    0.996641
76561198084389678    0.996403
Hyeeeon              0.996399
76561198059867666    0.996372
Name: --000--, dtype: float64

In [ ]:
similares_user.index.to_list()

['76561198120465213',
 '76561198076299728',
 '76561198084389678',
 'Hyeeeon',
 '76561198059867666']

In [ ]:
juegos_user = res.loc[user]
juegos_user[juegos_user > 0].index

Index([   550,    730,   1250,   4000,   9480,  24240,  55230, 104700, 104900,
       105600, 113400, 200210, 204360, 209270, 209870, 212800, 218620, 222900,
       225640, 227940, 230410, 233190, 238460, 238960, 242720, 246280, 263760,
       265630, 270550, 280790, 301520, 302830, 304050, 304930, 316390, 333930,
       339610, 363970, 381210, 444090],
      dtype='int64', name='item_id')

In [ ]:
juegos_rec = {}
juegos_user = res.loc[user].index
print(juegos_user)
for usr in similares_user.index:
  for item, val in res.loc[usr].sort_values(ascending=False).head(5).items():
    if item not in juegos_user:
      if item in juegos_rec:
        juegos_rec[item] += val
      else:
        juegos_rec[item] = val
print(juegos_rec)

Index([    10,     20,     30,     40,     50,     60,     70,     80,    130,
          220,
       ...
       512540, 512630, 514520, 516040, 520550, 521340, 521430, 521570, 521990,
       527340],
      dtype='int64', name='item_id', length=3682)
{730: 23278.2, 304930: 228.70000000000002, 4000: 51.2, 550: 403.6, 304050: 95.9, 113400: 5.2, 322900: 0.0, 322920: 0.0, 113200: 142.0, 227300: 59.8, 218620: 123.2, 339610: 246.89999999999998, 51100: 161.5, 218230: 10.5}


In [ ]:
juegos_rec = pd.Series(juegos_rec).sort_values(ascending=False).head(5)
juegos_rec

730       23278.2
550         403.6
339610      246.9
304930      228.7
51100       161.5
dtype: float64

In [ ]:
final_recs = []
for item in juegos_rec.index:
  if item in juegos.item_id:
    final_recs.append(juegos[juegos.item_id == item].app_name.values[0])
  else: final_recs.append(item)
print(final_recs)

['Counter-Strike: Global Offensive', 'Left 4 Dead 2', 339610, 304930, 51100]
